In [1]:
from netgen.occ import *
from ngsolve import Mesh
from ngsolve.webgui import Draw

import sys
sys.path.append("../build/rigid_body")
import rigid_body

import pythreejs as p3

import numpy as np
from IPython.display import display
import time

In [2]:
# set up OCC CAD model
box = Box(Pnt(0,0,0), Pnt(1,1,1))
geo = OCCGeometry(box)
mesh = Mesh(geo.GenerateMesh(maxh=1))

In [3]:
def extract_vertices(mesh: Mesh):
    "extracts a p3js compatible vertex list from a netgen.occ Mesh"
    combined_vertices = [] # flat array of vertices of all faces
    all_vertices = [vert.point for vert in mesh.vertices] # all available vertices

    # throw all vertices of all faces into combined_vertices
    for face in mesh.faces:
        for vertex in face.vertices:
            point = all_vertices[vertex.nr]
            combined_vertices.append(point)
            
    return np.array(combined_vertices, dtype=np.float32)

In [4]:
view_width = 600
view_height = 400

# set up pythreejs 3d object
original_vertices = extract_vertices(mesh)
buffergeom = p3.BufferGeometry(attributes = {"position" : p3.BufferAttribute(original_vertices, normalized=False)})
material = p3.MeshPhongMaterial(color='#ff3333', shininess=150, morphTargets=True, side="DoubleSide")
p3mesh = p3.Mesh(buffergeom, material, position=(0,0,0))

# extra scene contents
camera = p3.PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = p3.DirectionalLight(position=[0, 10, 10])
ambient_light = p3.AmbientLight()

# set up scene
scene = p3.Scene(children=[p3mesh, camera, key_light, ambient_light])
controller = p3.OrbitControls(controlling=camera)
renderer = p3.Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height, antialias=True) # if performance is bad, try removing antalias=True

In [5]:
display(renderer)

current_vertices = original_vertices

# set up simulation object
# rbody = rigid_body.RigidBody()

for i in range(10000):
    t = i/30

    # update vertex positions
    # rigid_body.update(original_vertices.data, t)
    current_vertices[:,0] += 0.1
    p3mesh.geometry = p3.BufferGeometry(attributes = {"position" : p3.BufferAttribute(current_vertices, normalized=False)})

    time.sleep(1/30)

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

KeyboardInterrupt: 